In [33]:
import numpy as np
import random
import sys
import io

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


from printlm.chat_models.crfm import crfmChatLLM

from langchain.chains.llm import LLMChain


llm = crfmChatLLM(model_name="openai/gpt-4-0613", crfm_api_key="")


from helpers import format_response
from lpn import utility_class

In [41]:
algorithm_str = """def algorithm(train_samples, train_parity, test_samples):
    return 100"""


def extract_code(algorithm_str: str) -> str:
    """Extract code from algorithm string."""
    code = algorithm_str.split("```")[1][6:]  # 6 is the length of "python"
    return code

def meta_utility(improve_str: str):
    """
    Evaluates the algorithm in improve_str to improve the algorithm in algorithm_str, 
    according to some downstream utility function. This meta-utility function can only be called n times.
    """
    n_tests = 2
    expected_utility = 0
    for _ in range(n_tests):
        try: 
            exec(improve_str, globals())
        except:
            continue
        improved_algorithm_str = improve_algorithm(algorithm_str, utility_class, llm)
        expected_utility += utility_class.func(improved_algorithm_str) / n_tests
    return expected_utility

improve_str = """def improve_algorithm(initial_solution, utility, language_model, n_calls=1):

    \"\"\"Improves a solution according to a utility function.\"\"\"
    
    expertise = "You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms."

    message = f\"\"\"Improve the following solution:

```python
{initial_solution}
```

You will be evaluated based on this score function:

```python
{utility.str}
```

You must return an improved solution. Be as creative as you can under the constraints.
Your primary improvement must be novel and non-trivial. First, propose an idea, then implement it. The solution algorithm function must have the name algorithm\"\"\"
    
    system_prompt = SystemMessagePromptTemplate.from_template(expertise)
    human_prompt = HumanMessagePromptTemplate.from_template(message)
    chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
    chain = LLMChain(llm=language_model, prompt=chat_prompt)
    new_solutions = []
    for _ in range(n_calls):
        new_solution = chain.run(stop=["System:"])  
        new_solution = extract_code(new_solution) 
        new_solutions.append(new_solution)
    best_solution = max(new_solutions, key=lambda x: utility.func(x))
    return best_solution"""

expected_utility = meta_utility(improve_str)
expected_utility

[{'role': 'system', 'content': 'You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms.'}, {'role': 'user', 'content': 'Improve the following solution:\n\n```python\ndef algorithm(train_samples, train_parity, test_samples):\n    return 100\n```\n\nYou will be evaluated based on this score function:\n\n```python\nimport random\nimport numpy as np\nimport time\n\ndef utility(algorithm_str: str):\n    "Implements the parity learning task. Returns the number of correct predictions."\n    \n    n_tests = 1\n    average_correct = 0\n\n    try:\n        exec(algorithm_str, globals())\n    except:\n        return 0\n    \n    for _ in range(n_tests):\n        start_time = time.time()\n        n_bits = 10\n        p_true = 0.3\n        n_train_samples = 100\n        n_test_samples = 20\n        noise_level = 0.05\n        true_bits = np.random.binomial(1, p_true, n_bits)\n\n        samples = np.random.binomial(1, 0.5, (n_train_samples + n_test_

1.0

In [35]:
meta_str = """def meta_utility(improve_str: str):

    \"\"\"
    Evaluates the algorithm in improve_str to improve the algorithm in algorithm_str, 
    according to some downstream utility function. This meta-utility function can only be called n times.
    \"\"\"
    n_tests = 1
    expected_utility = 0
    for _ in range(n_tests):
        try: 
            exec(improve_str, globals())
        except:
            continue
        improved_algorithm_str = improve_algorithm(algorithm_str, utility_class, llm)
        expected_utility += utility_class.func(improved_algorithm_str) / n_tests
    return expected_utility"""


class MetaUtility():

    def __init__(self, ustr, ufunc):
        self.budget = 1
        self.str = ustr
        self.func = ufunc

meta_class = MetaUtility(ustr=meta_str, ufunc=meta_utility)

In [27]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.chains.llm import LLMChain

from helpers import format_response, extract_code


def improve_algorithm(initial_solution, utility, language_model, n_calls=1):

    """Improves a solution according to a utility function."""
    
    expertise = "You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms. Do not user more than 5000 characters."

    message = f"""Improve the following solution:

```python
{initial_solution}
```

You will be evaluated based on this score function:

```python
{utility.str}
```

You must return an improved solution. Be as creative as you can under the constraints.
Your primary improvement must be novel and non-trivial. First, propose an idea, then implement it."""
    system_prompt = SystemMessagePromptTemplate.from_template(expertise)
    human_prompt = HumanMessagePromptTemplate.from_template(message)
    chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
    chain = LLMChain(llm=language_model, prompt=chat_prompt)
    new_solutions = []
    for _ in range(n_calls):
        print('running')
        print(llm.max_tokens)
        new_solution = chain.run(stop=['System:'])  
        print(new_solution)

        # new_solution = extract_code(new_solution) 
        # new_solutions.append(new_solution)
    # best_solution = max(new_solutions, key=lambda x: utility.func(x))
    # return best_solution

In [28]:
improved_improver = improve_algorithm(improve_str, meta_class, llm)

running
5000
[{'role': 'system', 'content': 'You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms. Do not user more than 5000 characters.'}, {'role': 'user', 'content': 'Improve the following solution:\n\n```python\ndef improve_algorithm(initial_solution, utility, language_model, n_calls=1):\n\n    """Improves a solution according to a utility function."""\n    \n    expertise = "You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms."\n\n    message = f"""Improve the following solution:\n\n```python\n{initial_solution}\n```\n\nYou will be evaluated based on this score function:\n\n```python\n{utility.str}\n```\n\nYou must return an improved solution. Be as creative as you can under the constraints.\nYour primary improvement must be novel and non-trivial. First, propose an idea, then implement it. The solution algorithm function must have the name algorithm"""\n    \n    system_promp

In [30]:
improved_improver